In [1]:
import numpy as np
import sys

from platform import python_version
print(python_version())


3.6.4


In [6]:
def align_sequences(seq1,seq2,edit_distance_matrix,indel_char = '-'):
    
    """
    This function computes an alignment, not necessarily unique,
    of the two sequence seq1 and seq2 corresponding to their 
    edit distance. It does so by backtracing a path through the 
    edit_distance_matrix input by checking logical conditions
    in a neighborhood of current matrix cell. The specific
    alignment produced by the code depends on the order of the 
    conditionals in the code.
    
    inputs:
    
    seq1: first sequence
    
    seq 1: second sequence
    
    edit_distance_matrix: a (# symbols in seq1 + 1) X (# symbols in seq2 + 1) matrix
    of edit distances, with the value at row i, column j corresponding to
    the edit distance between the (i - 1)th prefix of seq1 and the
    (j - 1)th prefix of seq2 (with i from 1 to len(seq1) and j from 1 to len(seq2),
    and with 0-based indexing of seq1 and seq2).
    
    indel_char: character to be used to represent insertions/deletions
    
    outputs:
    
    aligned_seq1: seq1 transformed to be aligned with seq2 in a manner consistent with the
    edit distance

    aligned_seq2: seq2 transformed to be aligned with seq1 in a manner consistent with the
    edit distance
    
    """
    
    num_rows = edit_distance_matrix.shape[0]
    num_cols = edit_distance_matrix.shape[1]

    aligned_seq1 = ''
    aligned_seq2 = ''
    
    row_index = num_rows - 1
    col_index = num_cols - 1
    
    while (row_index > 0 or col_index > 0):

        #note that the row and column indices of edit_distance_matrix are advanced
        #by 1 relative to the corresponding indices of seq1 and seq2, respectively.
        seq1_index = row_index - 1
        seq2_index = col_index - 1

        if ((seq1[seq1_index] == seq2[seq2_index]) and \
            (edit_distance_matrix[row_index,col_index] == edit_distance_matrix[row_index - 1,col_index - 1])):

            #match
            aligned_seq1 += seq1[seq1_index]

            aligned_seq2 += seq2[seq2_index]

            row_index -= 1

            col_index -= 1
        
        #mismatch
        elif (edit_distance_matrix[row_index,col_index] == edit_distance_matrix[row_index - 1,col_index - 1] + 1):

            aligned_seq1 += seq1[seq1_index]

            aligned_seq2 += seq2[seq2_index]

            row_index -= 1

            col_index -= 1

        #deletion
        elif (edit_distance_matrix[row_index,col_index] == edit_distance_matrix[row_index - 1,col_index] + 1):

            aligned_seq1 += seq1[seq1_index]

            aligned_seq2 += indel_char

            row_index -= 1

        #insertion
        elif (edit_distance_matrix[row_index,col_index] == edit_distance_matrix[row_index ,col_index - 1] + 1):

            aligned_seq1 += indel_char

            aligned_seq2 += seq2[seq2_index]

            col_index -= 1

        #unexpected case
        else:

            print("Unexpected alignment case.")

            return '',''

    aligned_seq1 = aligned_seq1[::-1]
    
    aligned_seq2 = aligned_seq2[::-1]
        
    return aligned_seq1,aligned_seq2
    

def compute_edit_distance(seq1,seq2):
    
    """
    Edit distance is the minimum number of insertions, deletions,
    and substitution of symbols required to transform one sequence
    into another.
    
    This function uses dynamic programming to compute the edit distance
    between the sequences seq1 and seq2. It also returns the aligned
    sequences, with a dummy character used in aligned seq1 and
    aligned seq2 to represent insertions and deletions, respectively.
    """
    
    len_seq1 = len(seq1)
    len_seq2 = len(seq2)
    
    #initialize edit distance matrix with zeros
    edit_distance_matrix =\
        np.zeros(shape = (len_seq1 + 1,len_seq2 + 1),\
        dtype = np.int32,order = 'C')
    
    #populate first column of edit distance matrix
    #with row index to represent consecutive deletions
    for row_index in range(len_seq1 + 1):
        edit_distance_matrix[row_index,0] = row_index
        
    #populate first row of edit distance matrix
    #with column index to represent consecutive insertions
    for col_index in range(len_seq2 + 1):
        edit_distance_matrix[0,col_index] = col_index
        
    #populate edit distance matrix
    for col_index in range(1,len_seq2 + 1):
        for row_index in range(1,len_seq1 + 1):
            
            distance_for_insertion = edit_distance_matrix[row_index,col_index - 1] + 1
            
            distance_for_deletion = edit_distance_matrix[row_index - 1,col_index] + 1
            
            #note that the row and column indices of edit_distance_matrix are advanced
            #by 1 relative to the corresponding indices of seq1 and seq2, respectively.
            seq1_index = row_index - 1
            seq2_index = col_index - 1
            
            if (seq1[seq1_index] == seq2[seq2_index]):
                
                #current distance if there is a symbol match
                distance_for_mismatch_or_match = edit_distance_matrix[row_index - 1,col_index - 1]
            
            else:
            
                #current distance if there is a symbol mismatch
                distance_for_mismatch_or_match = edit_distance_matrix[row_index - 1,col_index - 1] + 1
            
            edit_distance_matrix[row_index,col_index] = \
            min(distance_for_insertion,\
                distance_for_deletion,\
                distance_for_mismatch_or_match)
            
    edit_distance = edit_distance_matrix[len_seq1,len_seq2]
    
    aligned_seq1,aligned_seq2 = align_sequences(seq1,seq2,edit_distance_matrix)
    
    return edit_distance,aligned_seq1,aligned_seq2


def edit_distance_wrapper(seq1,seq2):

    edit_distance,aligned_seq1,aligned_seq2 = compute_edit_distance(seq1,seq2)
    print("sequence 1 = {:s}".format(seq1))
    print("sequence 2 = {:s}".format(seq2))
    print('\nedit distance between sequences 1 and 2 = {:d}'.format(edit_distance))
    print('\naligned sequences:\n{:s}\n{:s}'.format(aligned_seq1,aligned_seq2))

    return

In [7]:
#EXAMPLE 1

seq1 = 'editing'
seq2 = 'distance'

edit_distance_wrapper(seq1,seq2)



sequence 1 = editing
sequence 2 = distance

edit distance between sequences 1 and 2 = 5

aligned sequences:
edi-tin-g
-distance


In [8]:
#EXAMPLE 2

seq1 = 'The small brown squirrel gnawed on a nut.'
seq2 = 'The small gray squirrel gnawed on an acorn.'

edit_distance_wrapper(seq1,seq2)

sequence 1 = The small brown squirrel gnawed on a nut.
sequence 2 = The small gray squirrel gnawed on an acorn.

edit distance between sequences 1 and 2 = 10

aligned sequences:
The small brown squirrel gnawed on a- --nut.
The small gr-ay squirrel gnawed on an acorn.


In [9]:
#EXAMPLE 3

seq1 = 'AAAC'
seq2 = 'AGC'

edit_distance_wrapper(seq1,seq2)

sequence 1 = AAAC
sequence 2 = AGC

edit distance between sequences 1 and 2 = 2

aligned sequences:
AAAC
-AGC
